In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
import sys
sys.path.insert(0, "../fairseq")
sys.path.insert(0, "../")
sys.path.insert(0, "~/utility/imputer-pytorch")
# %load_ext autoreload
# %autoreload 2

In [ ]:
# https://github.com/pytorch/fairseq/blob/master/fairseq_cli/generate.py

In [ ]:
from fairseq import (
    checkpoint_utils,
    options,
    quantization_utils,
    tasks,
    utils,
)
from torchinfo import summary
import logging
import os
import matplotlib.pyplot as plt
import torch
from torch_imputer import best_alignment

In [ ]:
checkpoint="../exp/checkpoints/toy_attn_sort_ctc/checkpoint_best.pt"
inference_config_yaml="../exp/infer_toy.yaml"
use_cuda = True

states = checkpoint_utils.load_checkpoint_to_cpu(
    path=checkpoint, arg_overrides=None, load_on_all_ranks=False)
cfg = states["cfg"]
cfg.task.inference_config_yaml = inference_config_yaml

In [ ]:
logging.basicConfig(
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=os.environ.get("LOGLEVEL", "INFO").upper(),
    stream=sys.stdout,
)
logger = logging.getLogger("fairseq_cli.train")

In [ ]:
utils.import_user_module(cfg.common)

# Setup task, e.g., translation, language modeling, etc.
task = tasks.setup_task(cfg.task)
# Build model and criterion
model = task.build_model(cfg.model)
criterion = task.build_criterion(cfg.criterion)
logger.info(summary(model))
logger.info("task: {}".format(task.__class__.__name__))
logger.info("model: {}".format(model.__class__.__name__))
logger.info("criterion: {}".format(criterion.__class__.__name__))

In [ ]:
logger.info("loading model(s) from {}".format(cfg.common_eval.path))
model = task.build_model(cfg.model)
model.load_state_dict(
    states["model"], strict=True, model_cfg=cfg.model
)

# Optimize ensemble for generation
if use_cuda:
    model.half()
    model.cuda()
model.prepare_for_inference_(cfg)

In [ ]:
task.load_dataset(cfg.dataset.gen_subset, task_cfg=cfg.task)

In [ ]:
# Load dataset 
itr = task.get_batch_iterator(
    dataset=task.dataset(cfg.dataset.gen_subset),
    max_tokens=cfg.dataset.max_tokens,
    max_sentences=cfg.dataset.batch_size,
    max_positions=utils.resolve_max_positions(
        task.max_positions(), model.max_positions() #*[m.max_positions() for m in models]
    ),
    ignore_invalid_inputs=cfg.dataset.skip_invalid_size_inputs_valid_test,
    required_batch_size_multiple=cfg.dataset.required_batch_size_multiple,
    seed=cfg.common.seed,
    num_shards=cfg.distributed_training.distributed_world_size,
    shard_id=cfg.distributed_training.distributed_rank,
    num_workers=cfg.dataset.num_workers,
    data_buffer_size=cfg.dataset.data_buffer_size,
).next_epoch_itr(shuffle=False)

generator = task.build_generator(
    [model], cfg.generation, extra_gen_cls_kwargs=None
)

# Handle tokenization and BPE
def decode(
    tensor,
    escape_unk=False,
    unk_string=None,
    include_eos=False,
    separator=None,
):    
    def token_string(i):
        if i == task.tgt_dict.unk():
            if unk_string is not None:
                return unk_string
            else:
                return task.tgt_dict.unk_string(escape_unk)
        else:
            return task.tgt_dict[i]

    sent = [token_string(i) for i in tensor]
    return sent if separator is None else separator.join(sent)

In [ ]:
for sample in itr:
    if use_cuda:
        sample = utils.move_to_cuda(sample) 
    if "net_input" not in sample:
        continue

    break

src_tokens = sample["net_input"]["src_tokens"]
src_lengths = sample["net_input"]["src_lengths"]
target = sample["target"]
logits, extra = model.forward(src_tokens, src_lengths, return_all_hiddens=True)
pred_argmax = logits.argmax(-1)

In [ ]:
net_output = (logits, extra)
lprobs = model.get_normalized_probs(
    net_output, log_probs=True
)
bsz = target.size(0)
max_src = lprobs.size(1)
lprobs = lprobs.transpose(1, 0).contiguous()

# get subsampling padding mask & lengths
if net_output[1]["padding_mask"] is not None:
    non_padding_mask = ~net_output[1]["padding_mask"]
    input_lengths = non_padding_mask.long().sum(-1)
else:
    input_lengths = lprobs.new_ones(
        (bsz, max_src), dtype=torch.long).sum(-1)

pad_mask = (target != task.tgt_dict.pad()) & (
    target != task.tgt_dict.eos()
)
targets_flat = target.masked_select(pad_mask)
target_lengths = pad_mask.long().sum(-1)

forced_states = best_alignment(
    lprobs, target, input_lengths, target_lengths, blank=0, zero_infinity=False
)

In [ ]:
pred_argmax = [
    [0 if state % 2 == 0 else sent[state // 2] for state in states]
    for states, sent in zip(forced_states, target)
]

In [ ]:
from typing import Optional, Callable

def plot_align(
    aln,
    titles: Optional[list]=None,
    x_tokens: Optional[list]=None,
    y_tokens: Optional[list]=None,
    norm_fn: Optional[Callable]=lambda x:x,
    columns: Optional[int]=1,
    tick_size: Optional[int]=10,
    fig_size: Optional[tuple]=(12,8),
    save: Optional[str]=None,
    cmap=plt.cm.viridis #plt.cm.Blues,
):
    """Function to plot the alignment with tokens
    """
    
    n_graphs = len(aln)
    rows = 1+(n_graphs // columns)
    ylen, xlen = aln[0].shape
    
    if titles is None:
        titles = [f"example {i}" for i in range(n_graphs)]
    if x_tokens is None:
        x_tokens = [range(xlen)]*n_graphs
    if y_tokens is None:
        y_tokens = [range(ylen)]*n_graphs    
    
    fig = plt.figure(figsize=(fig_size[0]*columns,fig_size[1]*rows), dpi=100) 
    for i,a in enumerate(aln):
        ax = fig.add_subplot(rows, columns, i+1)
        ax.imshow(norm_fn(a), cmap=cmap)
        ax.set_title(titles[i]) # title
        for tick in ax.get_xticklabels(): # diagonal xtick
            tick.set_rotation(45)
        ax.set_xticks(range(xlen)) 
        ax.set_xticklabels(x_tokens[i%len(x_tokens)], fontsize=tick_size)
        ax.set_yticks(range(ylen)) 
        ax.set_yticklabels(y_tokens[i%len(y_tokens)], fontsize=tick_size)
     
    if save is not None:
        fig.savefig(save)
        
def get_alphabet():
    """ credit goes to 
    https://stackoverflow.com/questions/1477294/generate-random-utf-8-string-in-python """

    try:
        get_char = unichr
    except NameError:
        get_char = chr

    include_ranges = [
        (0x0021, 0x0021),
        (0x0023, 0x0026),
        (0x0028, 0x007E),
        (0x00A1, 0x00AC),
        (0x00AE, 0x00FF),
        (0x0100, 0x017F),
        (0x0180, 0x024F),
        (0x2C60, 0x2C7F),
        (0x16A0, 0x16F0),
        (0x0370, 0x0377),
        (0x037A, 0x037E),
        (0x0384, 0x038A),
        (0x038C, 0x038C),
    ]

    alphabet = [
        get_char(code_point) for current_range in include_ranges
        for code_point in range(current_range[0], current_range[1] + 1)
    ]
    return alphabet

def chrs2ids(chrs, otype=str):
    alphabet = get_alphabet()
    return [otype(alphabet.index(c)) if c in alphabet else otype(c) for c in chrs]

In [ ]:
N = 1

x_tokens = [chrs2ids(decode(t)) for t in src_tokens[:N]]
# y_tokens = [decode(t) for t in target[:N]]
y_tokens = [decode(t) for t in pred_argmax[:N]]

attn_weights = extra["attn"][0].data.float().cpu()
log_alpha = extra["log_alpha"][0].data.float().cpu()
b, n = attn_weights.shape[:2]
plot_align(
    torch.stack((attn_weights, -log_alpha), dim=1).view(2*b, n, n)[:2*N],
    fig_size=(13,13),
    x_tokens=x_tokens,
    y_tokens=y_tokens,
    titles = ["transport map", "cost"],
)

In [ ]:
from simultaneous_translation.modules.sinkhorn_attention import (
    gumbel_sinkhorn
)

# encoder_states = extra["encoder_out"]["encoder_states"]
# non_causal_states = encoder_states[-2].transpose(1,0)
# causal_states = encoder_states[-4].transpose(1,0)

# causal outputs
causal_out = model.encoder.forward_causal(src_tokens, src_lengths)
causal_states = x = causal_out["encoder_out"][0]
encoder_padding_mask = causal_out["encoder_padding_mask"][0] \
    if len(causal_out["encoder_padding_mask"]) > 0 else None
encoder_states = causal_out["encoder_states"]

# forward non-causal layers
for layer in model.encoder.non_causal_layers:
    x = layer(x, encoder_padding_mask)
non_causal_states = x

# reorder using sinkhorn layers
_, attn, log_alpha = model.encoder.sinkhorn_layer(
    query=x,  # this is non_causal_states
    key=causal_states,
    value=causal_states,
    key_padding_mask=encoder_padding_mask,
)

# q = x.transpose(0, 1)
# k = causal_states.transpose(0, 1)

# energy_fn = "l2"
# if energy_fn == "dot":
#     attn_weights = torch.bmm(q, k.transpose(1, 2))
# elif energy_fn == "cos":
#     # serious underflow for half.
#     attn_weights = torch.cosine_similarity(
#         q.float().unsqueeze(2),  # (bsz, tgt_len, 1, embed_dim)
#         k.float().unsqueeze(1),  # (bsz, 1, src_len, embed_dim)
#         dim=-1,
#     )
# elif energy_fn == "l2":
#     # cdist not inplemented for half.
#     attn_weights = -torch.cdist(q.float(), k.float(), p=2)
# attn_weights = attn_weights.type_as(q)

# log_alpha = attn_weights.clone()
# attn = gumbel_sinkhorn(
#     attn_weights.float(),
#     tau=0.75,
#     n_iter=8,
#     noise_factor=0
# )

attn_weights = attn.data.float().cpu()
log_alpha = log_alpha.data.float().cpu()

In [ ]:
attn_weights = gumbel_sinkhorn(
    log_alpha,
    tau=0.1,
    n_iter=8,
    noise_factor=0.
)
b, n = attn_weights.shape[:2]
plot_align(
    torch.stack((attn_weights, -log_alpha), dim=1).view(2*b, n, n)[:2*N],
    fig_size=(13,13),
    x_tokens=x_tokens,
    y_tokens=y_tokens,
    titles = ["transport map", "cost"],
)

In [ ]:
def real_sinkhorn(log_alpha: torch.Tensor, tau: float = 0.7, n_iter: int = 20, noise: bool = True):
    """
    https://github.com/gpeyre/SinkhornAutoDiff/blob/master/sinkhorn_pointcloud.py
    """
    b, _, n = log_alpha.size()
    mu = log_alpha.new_ones((b, n)) / n
    nu = log_alpha.new_ones((b, n)) / n
    epsilon = tau
    C = -log_alpha
    
    # Parameters of the Sinkhorn algorithm.
    rho = 1  # (.5) **2          # unbalanced transport
    tau = -.8  # nesterov-like acceleration
    lam = rho / (rho + epsilon)  # Update exponent
    thresh = 10**(-1)  # stopping criterion

    # Elementary operations .....................................................................
    def ave(u, u1):
        "Barycenter subroutine, used by kinetic acceleration through extrapolation."
        return tau * u + (1 - tau) * u1

    def M(u, v):
        "Modified cost for logarithmic updates"
        "$M_{ij} = (-c_{ij} + u_i + v_j) / \epsilon$"
        return (-C + u.unsqueeze(-1) + v.unsqueeze(-2)) / epsilon

    def lse(A, dim):
        "log-sum-exp"
        return torch.logsumexp(A, dim, keepdim=True)
        #return torch.log(torch.exp(A).sum(1, keepdim=True) + 1e-6)  # add 10^-6 to prevent NaN

    # Actual Sinkhorn loop ......................................................................
    u, v, err = 0. * mu, 0. * nu, 0.
    actual_nits = 0  # to check if algorithm terminates because of threshold or max iterations reached

    for i in range(n_iter):
        u1 = u  # useful to check the update
        u = epsilon * (torch.log(mu) - lse(M(u, v), -1).squeeze()) + u
        v = epsilon * (torch.log(nu) - lse(M(u, v), -2).squeeze()) + v
        # accelerated unbalanced iterations
        # u = ave( u, lam * ( epsilon * ( torch.log(mu) - lse(M(u,v)).squeeze()   ) + u ) )
        # v = ave( v, lam * ( epsilon * ( torch.log(nu) - lse(M(u,v).t()).squeeze() ) + v ) )
        err = (u - u1).abs().sum()

        actual_nits += 1
        if (err < thresh).data.numpy():
            break
    U, V = u, v
    pi = torch.exp(M(U, V))  # Transport plan pi = diag(a)*K*diag(b)
    return pi